In [48]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [49]:
file_name = "NF-CSE-CIC-IDS2018-v2.csv"
# file_name = "NF-UNSW-NB15-v2.csv"
data = pd.read_csv(file_name)

In [50]:
data.Label.value_counts()

Label
0.0    4084171
1.0     554361
Name: count, dtype: int64

In [51]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [52]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [53]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS', nan], dtype=object)

In [54]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [55]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,408417,408417,408417,408417,408417,408417,408417,408417,408417,408417,...,408417,408417,408417,408417,408417,408417,408417,408417,408417,408417
Bot,3521,3521,3521,3521,3521,3521,3521,3521,3521,3521,...,3521,3521,3521,3521,3521,3521,3521,3521,3521,3521
Brute Force -Web,55,55,55,55,55,55,55,55,55,55,...,55,55,55,55,55,55,55,55,55,55
Brute Force -XSS,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23
DDOS attack-HOIC,26496,26496,26496,26496,26496,26496,26496,26496,26496,26496,...,26496,26496,26496,26496,26496,26496,26496,26496,26496,26496
DDOS attack-LOIC-UDP,48,48,48,48,48,48,48,48,48,48,...,48,48,48,48,48,48,48,48,48,48
DDoS attacks-LOIC-HTTP,7564,7564,7564,7564,7564,7564,7564,7564,7564,7564,...,7564,7564,7564,7564,7564,7564,7564,7564,7564,7564
DoS attacks-GoldenEye,694,694,694,694,694,694,694,694,694,694,...,694,694,694,694,694,694,694,694,694,694
DoS attacks-Hulk,10618,10618,10618,10618,10618,10618,10618,10618,10618,10618,...,10618,10618,10618,10618,10618,10618,10618,10618,10618,10618


In [56]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [57]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [58]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [59]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [60]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
3955721,172.31.67.42,172.31.0.2,6.467650e-09,5.167049e-09,0.000042,5.744228e-07,0.000118,5.744228e-07,6.493897e-09,6.493897e-09,...,0.000000,0.000000,6.789944e-08,6.789944e-08,5.804603e-08,5.040895e-09,0.000034,6.865008e-08,"[6.467649681641264e-09, 5.167049175319861e-09,...",3955721
2325016,66.17.123.196,172.31.65.27,1.033723e-08,4.729764e-10,0.000077,4.206477e-07,0.000098,3.680667e-07,1.452982e-10,7.759581e-09,...,0.000431,0.003365,6.215314e-09,6.215314e-09,1.032167e-08,1.032167e-08,0.000000,6.284025e-09,"[1.0337231687413666e-08, 4.729763781866915e-10...",2325016
1764450,172.31.64.121,23.45.134.221,3.585086e-08,1.640343e-09,0.000764,4.923654e-06,0.001081,4.194224e-06,1.363043e-08,3.653466e-09,...,0.001494,0.005325,8.075172e-10,8.075172e-10,3.579688e-08,3.579688e-08,0.000000,2.179381e-08,"[3.5850855809563285e-08, 1.6403432222909793e-0...",1764450
3566283,172.31.66.66,23.217.40.12,3.471685e-09,1.588457e-10,0.000014,1.765894e-07,0.000124,1.765894e-07,2.744133e-09,8.087445e-11,...,0.000145,0.000516,2.087368e-09,2.087368e-09,3.466457e-09,3.466457e-09,0.000000,2.110445e-09,"[3.4716846101506993e-09, 1.58845700935081e-10,...",3566283
3814722,178.137.217.167,172.31.66.97,1.316328e-07,1.778572e-09,0.000088,2.008674e-06,0.000088,2.008674e-06,1.566218e-10,1.531469e-10,...,0.010997,0.005485,7.914488e-08,7.914488e-08,1.314346e-07,1.314346e-07,0.000000,8.001984e-08,"[1.3163276546494432e-07, 1.778571631316055e-09...",3814722


In [61]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [ ]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
print(train_g) # 27201 nodes, 324697 edges
train_g = train_g.to_directed()
print(train_g) # 27201 nodes, 648528 edges (doubled because of directed graph)
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['edge_id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

MultiGraph with 27201 nodes and 324697 edges
MultiDiGraph with 27201 nodes and 648528 edges


In [63]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [64]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [65]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [66]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [67]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [68]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [69]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [70]:
# Convert to GPU
train_g = train_g.to('cuda')

In [ ]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [72]:
dgi.load_state_dict(torch.load('best_dgi_CSE.pkl'))

<All keys matched successfully>

In [73]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [74]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [75]:
# Convert to GPU
test_g = test_g.to('cuda')

In [76]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [ ]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [78]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [79]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [80]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,0.041696,0.038112,-0.007873,0.058345,-0.012183,-0.001185,0.025321,0.026215,0.024554,0.012352,...,0.000000e+00,0.000000e+00,0.000000,0.000000,1.194808e-07,1.194808e-07,9.690518e-08,8.870327e-09,0.000016,1.208017e-07
1,0.041696,0.038112,-0.007873,0.058345,-0.012183,-0.001185,0.025321,0.026215,0.024554,0.012352,...,0.000000e+00,0.000000e+00,0.000000,0.000000,7.035671e-08,7.035671e-08,6.187932e-08,5.223324e-09,0.000008,7.113452e-08
2,0.041696,0.038112,-0.007873,0.058345,-0.012183,-0.001185,0.025321,0.026215,0.024554,0.012352,...,0.000000e+00,0.000000e+00,0.000000,0.000000,2.713102e-08,2.713102e-08,2.353985e-08,2.014223e-09,0.000005,2.743096e-08
3,0.041696,0.038112,-0.007873,0.058345,-0.012183,-0.001185,0.025321,0.026215,0.024554,0.012352,...,0.000000e+00,0.000000e+00,0.000000,0.000000,8.361676e-08,8.361676e-08,7.148248e-08,6.111130e-09,0.000042,8.454116e-08
4,0.041696,0.038112,-0.007873,0.058345,-0.012183,-0.001185,0.025321,0.026215,0.024554,0.012352,...,0.000000e+00,0.000000e+00,0.000000,0.000000,1.299508e-07,1.299508e-07,1.127500e-07,9.647622e-09,0.000003,1.313874e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277921,0.134202,0.013851,-0.009862,0.021284,0.026228,0.035215,0.080903,0.023398,0.035047,0.037234,...,1.842810e-07,7.371239e-07,0.001510,0.004954,1.071401e-09,1.071401e-09,3.617444e-08,3.617444e-08,0.000000,2.202368e-08
277922,0.045252,0.042160,0.001124,0.051484,-0.019799,-0.008183,0.016639,-0.001401,0.025258,-0.002944,...,0.000000e+00,0.000000e+00,0.003025,0.001509,2.177201e-08,2.177201e-08,3.615641e-08,3.615641e-08,0.000000,2.201270e-08
277923,0.043131,0.044241,-0.016499,0.056850,0.000636,-0.010170,0.038870,0.046780,0.023435,0.019008,...,0.000000e+00,5.488923e-08,0.000225,0.000738,3.244081e-09,3.244081e-09,5.387391e-09,5.387391e-09,0.000000,3.279945e-09
277924,0.044904,0.039963,-0.000768,0.051340,-0.017594,-0.003586,0.017617,0.004360,0.029578,0.002643,...,0.000000e+00,0.000000e+00,0.011180,0.005485,7.914472e-08,7.914472e-08,1.314343e-07,1.314343e-07,0.000000,8.001968e-08


In [81]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [82]:
from pyod.models.cblof import CBLOF
n_est = [4,5,6,7,9,10] # can't be lower than 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [06:43<00:00, 11.22s/it]


{'n_estimators': 6, 'con': 0.001}
0.9476780551816544
              precision    recall  f1-score   support

         0.0     0.9783    0.9988    0.9884    244664
         1.0     0.9896    0.8370    0.9069     33262

    accuracy                         0.9794    277926
   macro avg     0.9839    0.9179    0.9477    277926
weighted avg     0.9796    0.9794    0.9787    277926



In [83]:
n_est = [4,5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [06:05<00:00, 10.15s/it]


{'n_estimators': 9, 'con': 0.1}
0.7450266428447414
              precision    recall  f1-score   support

         0.0     0.9336    0.9546    0.9440    244664
         1.0     0.6001    0.5009    0.5460     33262

    accuracy                         0.9003    277926
   macro avg     0.7669    0.7278    0.7450    277926
weighted avg     0.8937    0.9003    0.8964    277926



In [84]:
# HBOS  Embeddings+Raw (Multimodal)

In [85]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [86]:
from pyod.models.hbos import HBOS

n_est = [50,55,60,65,70,75,80]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 42/42 [05:59<00:00,  8.57s/it]


{'n_estimators': 70, 'con': 0.001}
0.9438422977526526
              precision    recall  f1-score   support

         0.0     0.9769    0.9986    0.9877    244664
         1.0     0.9880    0.8265    0.9000     33262

    accuracy                         0.9780    277926
   macro avg     0.9824    0.9125    0.9438    277926
weighted avg     0.9782    0.9780    0.9772    277926



In [87]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [05:05<00:00,  8.49s/it]


{'n_estimators': 10, 'con': 0.1}
0.9337457527321992
              precision    recall  f1-score   support

         0.0     0.9780    0.9921    0.9850    244664
         1.0     0.9346    0.8359    0.8825     33262

    accuracy                         0.9734    277926
   macro avg     0.9563    0.9140    0.9337    277926
weighted avg     0.9728    0.9734    0.9727    277926



In [88]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [89]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [04:03<00:00,  6.77s/it]


{'n_estimators': 5, 'con': 0.001}
0.9443736959703346
              precision    recall  f1-score   support

         0.0     0.9770    0.9988    0.9878    244664
         1.0     0.9897    0.8268    0.9010     33262

    accuracy                         0.9782    277926
   macro avg     0.9833    0.9128    0.9444    277926
weighted avg     0.9785    0.9782    0.9774    277926



In [90]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [04:10<00:00,  6.96s/it]


{'n_estimators': 5, 'con': 0.1}
0.9238509799871679
              precision    recall  f1-score   support

         0.0     0.9718    0.9952    0.9834    244664
         1.0     0.9573    0.7878    0.8643     33262

    accuracy                         0.9704    277926
   macro avg     0.9646    0.8915    0.9239    277926
weighted avg     0.9701    0.9704    0.9691    277926



In [91]:
##  IF  Emb+Raw

In [92]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [93]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]/home/kienho/miniforge3/envs/py3.9/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:01<00:40,  1.77s/it]/home/kienho/miniforge3/envs/py3.9/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:03<00:38,  1.76s/it]/home/kienho/miniforge3/envs/py3.9/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:05<00:36,  1.75s/it]/home/kienho/miniforge3/envs/py3.9/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 17%|█▋        | 4/24 [00:06<00:34,  1.74s/it]/home/kienho/miniforge3/en

{'n_estimators': 100, 'con': 0.01}
0.9400240961761546
              precision    recall  f1-score   support

         0.0     0.9845    0.9871    0.9858    244664
         1.0     0.9032    0.8855    0.8943     33262

    accuracy                         0.9749    277926
   macro avg     0.9438    0.9363    0.9400    277926
weighted avg     0.9747    0.9749    0.9748    277926



In [94]:
from sklearn.ensemble import IsolationForest

n_est = [50, 75, 100, 125, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [01:52<00:00,  3.76s/it]


{'n_estimators': 75, 'con': 0.1}
0.9050731045035528
              precision    recall  f1-score   support

         0.0     0.9718    0.9852    0.9784    244664
         1.0     0.8788    0.7894    0.8317     33262

    accuracy                         0.9618    277926
   macro avg     0.9253    0.8873    0.9051    277926
weighted avg     0.9606    0.9618    0.9609    277926

